In [1]:
# Cell 1

import pylab as pl
import casadi as ca
import numpy as np
import pandas
print(ca.__version__)

3.3.0


In [2]:
# Cell 2

pl.close("all")

# Parameters for experiments

## Duration and time points

hours = 24
controls_actions_per_hour = 1

t0 = 0.0;
tf = hours * 3600.0;
N = hours * controls_actions_per_hour;

time_points = pl.linspace(t0, tf, N + 1)

dt = (tf - t0) / N  # Duration of a time interval

## Number of storage layers in the HOT tank
nlayer_htes = 9

## The layer at which the HOT tank is connected to the load
layer_load_h = 6

## Number of storage layers in the COLD tank
nlayer_ctes = 0

df = pandas.read_excel('../data/winter_load_data.xlsx')
#print the column names
print(df.columns)
##get the values for a given column
Pth_Load_Heat = df['Pth_Load_Heat'].values
Time = df['Time'].values
Pel_Load = df['Pel'].values
epex_price_buy = df['epex_price_buy'].values
epex_price_sell = df['epex_price_sell'].values

# Build the tracking thermal load series
Pth_Load_H_track = np.repeat(Pth_Load_Heat, controls_actions_per_hour)


Index(['Time', 'Pel', 'Pth_Load_Heat', 'epex_price_buy', 'epex_price_sell'], dtype='object')


In [3]:
N

24

In [4]:
# Cell 3

# Physical parameters of the system

## Component 1: CHP
Pmin_CHP_Th = 0 #kW_Th / defined in Oscar's Python file
Pmax_CHP_Th = 10 # Oscar
Pmin_CHP_El = 0 # Oscar 
Pmax_CHP_El = 5 #kW_El / Oscar
#Efficiencies 
Pel_eff_CHP = 0.3 # Oscar
Pth_eff_CHP = 0.59 # Oscar
# TODO: check the names, these are used in OM
Pth_CHP_Nominal = 10.2 # in kW
Pel_CHP_Nominal= 5.3 # in kW 
CHP_eta_Thermal= 0.59
CHP_eta_Electrical= 0.3
CHP_Fuel_LHV= 11.8 


## Component 2: HTES
# Pmin_COIL_Th = 0 # Oscar
# Pmax_COIL_Th = 5.8#kW_Th # Oscar
# Pth_eff_COIL = 0.95 # Oscar
Pel_COIL_Nominal = 5.8
#Storage Tank
HTES_Cap = 1.5 #m³ # Oscar
HTES_Warm = 75 + 273.15 #K Highest possible temperature in tank # Oscar
HTES_Cold = 35 + 273.15 #K Lowest possible temperature in tank # Oscar
T_HTES_init_0 = 50 +273.15#Initial temperature in the Tank K # Oscar
Q_HTES_init = 30 #kW Initial Power in Tank # Oscar
lambda_eff = 0.0015
kappa = 0.002
vdot_HP_HT2 = 2

# Dimensions of the tank #OpenModelica
h=2.196 
D=1.0 
t=0.0125
n=nlayer_htes
pi=3.14 

## Component 3: LOAD_H 
Pth_LOAD_H = 1 # read from CSV or could be kept 0 or 1 kWth 
T_LOAD_H_CC_FL = 35 # °C 
vdot_CC = 0.8
# do not use Pth_CC
#Pth_CC = # Pth_Load_Heat (from CSV data)

## Component 4: HX3
# Heat exchanger
A_HX3 = 0.52 # m²
U_HX3 = 2.22 # kW/ (k.m²)


## Component 5: HP
Pmin_HP_Th = 0 # Oscar
Pmax_HP_Th = 16#kW_Th # Oscar
#COP_HP = 4.45 # Oscar
T_HP_min = 10 #Temperature min to operate the HP. # Oscar

#v_dot_HP_HT_FL_Set = 2.79 # in OM the name is v_dot_HT_FL_Set
#v_dot_HP_MT_FL_Set = 2.45 # in OM the name is v_dot_MT_FL_Set
HX_loss = 3 # degC
vdot_HP_HT1 = 2.79


## Component 6: OC2
# v_dot_OC2= 4.76 #m³/h
# ## Outdoor coils 
# A_OC = 221.4 #m²
# U_OC = 37.4972 / 1000 # kW/(k.m²) 
# RPM_max = 900 # "Max RPM of the fan"
# RPM_real = 900
# Pel_max = 1.4 #W "Maximum electrical power consumption of OC"
# Pel_OC2 = Pel_max / ((RPM_max / RPM_real) ** 3.0)
# new 20.4.18
Range_OC = 2.5
Pel_OC = 1.4 #kW


## Component 7: AdCM
v_dot_AdCM_LT_Set= 1.7
v_dot_AdCM_MT_Set= 4.2
v_dot_AdCM_HT_Set= 1.3
SF = 0
# new 20.4.18
mdot_AdCM_HT = 0


## HX1-HX2
# Heat exchangers 1 and 2
A_HX12 = 7.83
A_HX1 = A_HX12
A_HX2 = A_HX12
U_HX12 = 2.151  
U_HX1 = U_HX12 
U_HX2 = U_HX12


## Component 8: OC1
v_dot_OC1 = 4.76

## Component 9: CTES

## Component 10: CCM
v_dot_CC_H = 0.2 # These values are for Climate Chamber or Load, also defined once above
T_CC_H_FL = 35
v_dot_CC_C =0.2
T_CC_C_FL =16

v_dot_CCM_MT_FL_Set = 2.62 # in OM the name is v_dot_MT_FL_Set
v_dot_CCM_LT_FL_Set = 2.45 # in OM the name is v_dot_LT_FL_Set

## Component 11: LOAD_C
Pth_LOAD_C = 1
T_LOAD_C_CC_FL = 16

## Component 12: OC3


## Component 11: LOAD_C
Pth_LOAD_C = 1
T_LOAD_C_CC_FL = 1

## Component 12: OC3

# Constants

"Thermodynamical properties"
LHV_FUEL = 42600 #Kj/Kg
ro_FUEL = 853.5 #Kg/m³
rho_water = 977.8 #Kg/M3 (Cesar's name: ro_w)
Cp_w = 4.180 #KJ/Kg.K


"Costs" 
#Electricity costs
# To add
#C_El_B= pl.array(epex_price_buy)
#C_El_S= pl.array(epex_price_sell)

#Fuel Costs
Fuel = 510 #eur/m³

#T_Amb is splitted into day and night temperatures
#T_Amb = 10 + 273.15 #K
T_A_N = 6 #Temperature at 0 in the night for the temp linspace
T_A_M = 15#Temperature at 16hrs in the evening for the temp linspace

T_HP_min = 10 #Temperature min to operate the HP.
T_HTES_init_0 = 50 +273.15#Initial temperature in the Tank K
Q_HTES_init = 30 #kW Initial Power in Tank


#Formulate the LP Problem
#Set the time Steps
#n= int(Time.size) + 1

#define the ambient temperature linear increase or decrease
T_Amb=[]
days = int(round((n-1)/24.0))


T_Amb_1 = pl.linspace(T_A_N, T_A_M, (16))
T_Amb_2 = pl.linspace(T_A_M, T_A_N, (24)-16)

T_amb = T_A_N

T_amb_K = 300 #arbitrary




In [5]:
# Cell 4

# Secondary parameters to be used in equations (with conversion)
# All quantities here need to be derived from earlier notions
# No constant definition in this cell

m_dot_CC_H = v_dot_CC_H*rho_water/3600
m_dot_CC_C = v_dot_CC_C*rho_water/3600
m_dot_AdCM_LT_Set = v_dot_AdCM_LT_Set*rho_water/3600
m_dot_AdCM_MT_Set = v_dot_AdCM_MT_Set*rho_water/3600
m_dot_AdCM_HT_Set = v_dot_AdCM_HT_Set*rho_water/3600
# NOTE: to not use the "Set" in these parameters

#m_dot_HP_HT_FL_Set = v_dot_HP_HT_FL_Set * rho_water / 3600
#m_dot_HP_MT_FL_Set = v_dot_HP_MT_FL_Set * rho_water / 3600
# NOTE: to remove the m_dot_HP_HT_FL_Set, m_dot_HP_MT_FL_Set, immediately use the right hand side, DONE
# change v_dot_HP_HT_FL_Set to v_dot_HP_HT1, DONE

# m_dot_OC1 = v_dot_OC1*rho_water/3600
# m_dot_OC2 = v_dot_OC2*rho_water/3600

m_dot_CCM_MT_FL_Set = v_dot_CCM_MT_FL_Set*rho_water/3600
m_dot_CCM_LT_FL_Set = v_dot_CCM_LT_FL_Set*rho_water/3600

## Parameters for a layer in the HTES
zi = h / n;
mi = pi * (D / 2) ** 2 * zi * rho_water;
di = D - 2 * t;
Aamb = pi * D * zi;
Alayer = pi * di ** 2 / 4;


In [6]:
# Some equations for intermediate quantities that are not real variables
# Can be eliminated by hand

# Pth_AdCM_LT = f(T_HTES_t, T_AdCM_MT_RL, T_CTES_t)
# COP_AdCM = f(T_HTES_t, T_AdCM_MT_RL, T_CTES_t)
# Pth_CCM_LT = 
# Pth_AdCM_HT =
# Pth_AdCM_MT = 
# Pth_CCM_MT

In [7]:
# Cell 5

# Declaration of variables

## Number of control variables
# Winter problem: CHP_Switch, HP_Switch, COIL_Switch (coil in the hot tank)
nu = 3

## Number of state variables
nx = 1 + nlayer_htes + nlayer_ctes

## Number of algebraic variables (depending on the way we define variables)
ny = 10

## States and controls
u = ca.SX.sym("u", nu) # Control
x = ca.SX.sym("x", nx ) # Differential states: temperatures in the tank
y = ca.SX.sym("y", ny) # Output variable

# Input variables
CHP_Switch  = u[0]
HP_Switch   = u[1]
COIL_Switch = u[2]
#AdCM_Switch = u[3]
#CCM_Switch  = u[4]

# Temp bounds for control
u_lbw = list(0*pl.ones(nu))
u_ubw = list(1*pl.ones(nu))


# now only use "Switch", not "ON_int"
# CHP_ON_int  = u[0]
# RevHP_HP_ON_int   = u[1]
# AdCM_ON_int = u[2]
# RevHP_CC_ON_int  = u[3]

# State variables
Pth_CHP = x[0]
T_HTES = [k for k in range(nlayer_htes)]
T_CTES = [k for k in range(nlayer_ctes)]
for k in range(nlayer_htes):
    T_HTES[k] = x[1+k]
for k in range(nlayer_ctes):
    T_CTES[k] = x[1+nlayer_htes+k]

# Add names to conform to OpenModelica variables
T_HTES_t = T_HTES[nlayer_htes-1]  # top layer
T_HTES_l = T_HTES[layer_load_h-1] # layer output to the hot load
T_HTES_b = T_HTES[0] # bottom layer

# Output variables. We will re-order the component y[i] from 0, 1, ...
k = 0  # to increase this index when defining variables' names
vdot_FUEL    = y[k]
# NOTE: to remove mdot_FUEL, replace by vdot_FUEL, DONE

## Component 1: CHP
#k = k+1
#Pth_CHP      = y[k]
# don't keep algebraic copy of Pth_CHP, only leave it in the state 
k = k+1
Pel_CHP      = y[k]
# k = k+1
# m_dot_CHP_set = y[k]
k = k+1
mdot_CHP     = y[k]
k = k+1
T_CHP_FL     = y[k]

## Component 2: HTES - now these are all state variables
#T_HTES_t     = y[7]
#T_HTES_l     = y[8]
#T_HTES_b     = y[9]

## Component 3: LOAD_H
k = k+1
mdot_LOAD_H  = y[k]
k = k+1
T_LOAD_H_FL  = y[k]
# remove this variable 20.4.18
# k = k+1
# T_LOAD_H_CC_RL = y[k]

## Component 4: HX3
# k = k+1
# T_HX3_FL     = y[k]

## Component 5: HP
k = k+1
mdot_HP_HT = y[k]
k = k+1
Pel_HP = y[k]
k = k+1
Pth_HP_HT = y[k]
# 20.4.18: not used in the simplified version
#k = k+1
#Pth_HP_MT = y[k]
k = k+1
T_HP_HT_FL   = y[k]
# 20.4.18: not used in the simplified version
# k = k+1
# T_HP_HT_RL   = y[k]
# k = k+1
# T_HP_MT_FL   = y[k]
# k = k+1
# T_HP_MT_RL   = y[k]

# ## Component 6: OC2
# k = k+1
# Pel_OC2 = y[k]
# k = k+1
# v_dot_OC2 = y[k]
# k = k+1
# T_OC2_FL  = y[k]
# k = k+1
# T_OC2_RL  = y[k]

# ## Component 7: AdCM
# k = k+1
# T_AdCM_HT_FL = y[k]
# k = k+1
# T_AdCM_MT_FL = y[k]
# k = k+1
# T_AdCM_MT_RL = y[k]
# k = k+1
# T_AdCM_LT_FL = y[k]
# Pel_AdCM = y[k]


# ## Component 8: OC1
# k = k+1
# Pel_OC1 = y[k]
# k = k+1
# v_dot_OC1 = y[k]
# k = k+1
# T_OC1_FL     = y[k]
# k = k+1
# T_OC1_RL     = y[k]

# ## Component 9: CTES
# k = k+1
# T_CTES_t     = y[k]
# k = k+1
# T_CTES_b     = y[k]

# ## Component 10: CCM
# k = k+1
# Pel_CCM = y[k]
# k = k+1
# Pth_CCM_LT = y[k]
# k = k+1
# Pth_CCM_MT = y[k]
# k = k+1
# T_CCM_LT_FL  = y[k]
# k = k+1
# T_CCM_MT_FL  = y[k]
# k = k+1
# T_LOAD_C_CC_RL = y[k]

# ## Component 11: LOAD_C
# k = k+1
# Pth_LOAD_C = y[k]
# k = k+1
# mdot_LOAD_C = y[k]
# k = k+1
# T_LOAD_C_FL = y[k]

# ## Component 12: OC3
# k = k+1
# Pel_OC3 = y[k]








#to_be_checked_and_added


In [8]:
k
# should put ny in the previous cell equal to k+1

9

In [9]:
# Cell 6

# Secondary variables
# v_dot_LOAD_H = mdot_LOAD_H / rho_water
# v_dot_LOAD_C = mdot_LOAD_C / rho_water
# v_dot_FUEL = mdot_FUEL / rho_water
# v_dot_CHP = mdot_CHP / rho_water


In [10]:
# Cell 7

# Initial values of variable at time t=0

## Initial states
Pth_CHP_t0 = 1

T_HTES_layer_top_t0 = 35.0
T_HTES_layer_bottom_t0 = 25.0
T_HTES_t0 = pl.linspace(T_HTES_layer_top_t0, T_HTES_layer_bottom_t0, nlayer_htes)

T_CTES_layer_top_t0 = 25.0
T_CTES_layer_bottom_t0 = 10.0
T_CTES_t0 = pl.linspace(T_CTES_layer_top_t0, T_CTES_layer_bottom_t0, nlayer_ctes)

x_0 = [Pth_CHP_t0] + list(T_HTES_t0) + list(T_CTES_t0)


## Initial controls

chp_status_init = 0.0 * pl.ones(time_points.size - 1)

u_init = chp_status_init # to check

# Winter mode: 3 controls
u_0 = [0, 0, 0]




In [11]:
# Cell 8

# Equations
# TODO: make the separate lists of equations from Open Modelica initiated model,
#       one file for differential equations, one file for algebraic equations

## Differential equations: xdot = f(x,z,u)
dxdt = []
## Use equations like:
# dxdt.append((1.0 / m_s_i) * status_CHP * (mdot_CHP_to_9 * T_out_CHP - mdot_9_to_8 * T_s[8])) # T_9_dot
# ...
# dxdt = ca.vertcat(*dxdt)

# For the power of the CHP, the first state:
# d(Pth_CHP)/dt * 560.794 + Pth_CHP = Pth_CHP_Nominal * CHP_Switch
dxdt.append((1/560.794) * Pth_CHP_Nominal * CHP_Switch - Pth_CHP)

# new 20.4.18
# combined for winter and summer (AdCM needs to be counted)
C1 = (mdot_CHP + mdot_HP_HT - mdot_AdCM_HT) * Cp_w
C2 = (mdot_CHP + mdot_HP_HT - mdot_AdCM_HT - mdot_LOAD_H) * Cp_w
a = ca.SX.sym("a", nlayer_htes) # Intermediate variable, stand for water coming from the top layer to the layer below
b = ca.SX.sym("b", nlayer_htes) # Intermediate variable, stand for water coming from the load layer to the layer below

# Top layer of HTES is indexed nlayer_htes-1, note that the first index is 0
k = nlayer_htes-1
# only for winter, when AdCM does not work
dxdt.append(1/(Cp_w * mi) * ( Alayer * lambda_eff * (T_HTES[k-1] - T_HTES[k]) / zi
                             + Cp_w * mdot_CHP * (T_CHP_FL - T_HTES[k])
                             + Cp_w * mdot_HP_HT * (T_HP_HT_FL - T_HTES[k])
                             - Aamb * kappa * (T_HTES[k] - T_amb_K)
                             + Pel_COIL_Nominal * COIL_Switch / nlayer_htes) )
# combined for winter and summer (AdCM needs to be counted)
# an = 0
# bn = T_HTES[nlayer_htes]-T_HTES[nlayer_htes-1]
# dxdt.append(1/(Cp_w * mi) * ( Alayer * lambda_eff * (T_HTES[k-1] - T_HTES[k]) / zi
#                              + Cp_w * mdot_CHP * (T_CHP_FL - T_HTES[k])
#                              + Cp_w * mdot_HP_HT * (T_HP_HT_FL - T_HTES[k])
#                              + C1 * (an+bn)/2 + ca.fabs(C1)*(an-bn)/2
#                              - Aamb * kappa * (T_HTES[k] - T_amb_K)
#                              + Pel_COIL_Nominal * COIL_Switch / nlayer_htes) )

# NOTE: here, mdot_CHP can be replaced by the equation defined in the algebraic part, hence reduce one algebraic var
# need to test whether reducing mdot_CHP can help to solve the problem

# Loop from the layer next-to-top to above-layer-output-to-load (index of layer-output-to-load: layer_load_h-1)
for k in range(nlayer_htes-2, layer_load_h-1, -1):
    dxdt.append(1/(Cp_w * mi) * ( Alayer * lambda_eff * (T_HTES[k+1] - 2*T_HTES[k] + T_HTES[k-1]) / zi
                                 + Cp_w * ((mdot_CHP + mdot_HP_HT) * (T_HTES[k+1] - T_HTES[k]))
                                 - Aamb * kappa * (T_HTES[k] - T_amb_K) 
                                 + Pel_COIL_Nominal * COIL_Switch / nlayer_htes) )
#     # combined for winter and summer (AdCM needs to be counted)
#     a[k] = T_HTES[k+1]-T_HTES[k]
#     b[k] = T_HTES[k]-T_HTES[k-1]
#     dxdt.append(1/(Cp_w * mi) * ( Alayer * lambda_eff * (T_HTES[k+1] - 2*T_HTES[k] + T_HTES[k-1]) / zi
#                                  + C1 * (a[k]+b[k])/2 + ca.fabs(C1)*(a[k]-b[k])/2
#                                  - Aamb * kappa * (T_HTES[k] - T_amb_K) 
#                                  + Pel_COIL_Nominal * COIL_Switch / nlayer_htes) )

# To deal with the possibility that there could be reverse flow from bottom to the layer_load_h, if mdot_LOAD_H > (mdot_CHP + mdot_HP_HT):
# Make use of this formula: (x-y)*(a+b)/2 + abs(x-y)*(a-b)/2 = 
#  = (x-y)*a, if x>=y (positive)
#  = (x-y)*b, if x<y (negative)
# Note for Casadi: basic Python functions like abs, max, min are replaced by fabs, fmax, fmin 
# https://groups.google.com/d/msgid/casadi-users/d441bbf3-888d-4cfe-848b-2c2b9efb29f6%40googlegroups.com?utm_medium=email&utm_source=footer

# At the layer that water coming to the load, with index in the Python list = layer_load_h-1, use the formula above,
#   with: x=mdot_LOAD_H, y=mdot_CHP + mdot_HP_HT, a=T_HTES[layer_load_h-2], b=T_HTES[layer_load_h-1]
# then we can represent the heat that the layer RECEIVES from the liquid coming from the layer below (+ or -):
#   (mdot_LOAD_H-mdot_CHP - mdot_HP_HT)*T_HTES[layer_load_h-2] > 0, if mdot_LOAD_H>mdot_CHP + mdot_HP_HT, i.e. receiving water flowing up
#   (mdot_LOAD_H-mdot_CHP - mdot_HP_HT))*T_HTES[layer_load_h-1] < 0, if mdot_LOAD_H<mdot_CHP + mdot_HP_HT, i.e. losing water flowing down
k = layer_load_h-1
# dxdt.append(1/(4.18 * mi) * ( Alayer * lambda_eff * (T_HTES[k+1] - 2*T_HTES[k] + T_HTES[k-1]) / zi
#                              + 4.18 * ((mdot_CHP + mdot_HP_HT) * T_HTES[k+1] - mdot_LOAD_H * T_LOAD_H_FL + (mdot_LOAD_H - mdot_CHP - mdot_HP_HT)*(T_HTES[k-1] + T_HTES[k])/2 
#                                        + ca.fabs(mdot_LOAD_H - mdot_CHP - mdot_HP_HT)*(T_HTES[k-1] - T_HTES[k])/2  )
#                              - Aamb * kappa * (T_HTES[k] - T_amb_K)
#                              + Pel_COIL_Nominal * COIL_Switch / nlayer_htes) )
# # # To check old equation, when C1>0:
# #                              + 4.18 * ((mdot_CHP + mdot_HP_HT) * T_HTES[k+1] - mdot_LOAD_H * T_LOAD_H_FL 
# #                                        + (mdot_LOAD_H - mdot_CHP - mdot_HP_HT)*(T_HTES[k-1] + T_HTES[k])/2 
# #                                        + ca.fabs(mdot_LOAD_H - mdot_CHP - mdot_HP_HT)*(T_HTES[k-1] - T_HTES[k])/2  )

# #                              + C1 * a[k] 
# #                              + C2 * (T_HTES[k]-T_HTES[k-1])/2 + ca.fabs(C2)*(-T_HTES[k]+T_HTES[k-1])/2
# #                         = Cp_w * (mdot_CHP + mdot_HP_HT) * (T_HTES[k+1]-T_HTES[k])
# #                         + Cp_w * (mdot_CHP + mdot_HP_HT - mdot_LOAD_H) * (T_HTES[k]-T_HTES[k-1])/2 
# #                         + ca.fabs(Cp_w * (mdot_CHP + mdot_HP_HT - mdot_LOAD_H))*(T_HTES[k-1] - T_HTES[k])/2


# #                         = Cp_w * (mdot_CHP + mdot_HP_HT) * T_HTES[k+1]
# #                         - Cp_w * (mdot_CHP + mdot_HP_HT) * T_HTES[k]
# #                         - Cp_w * (mdot_LOAD_H - mdot_CHP - mdot_HP_HT) * (T_HTES[k]-T_HTES[k-1])/2 

dxdt.append(1/(4.18 * mi) * ( Alayer * lambda_eff * (T_HTES[k+1] - 2*T_HTES[k] + T_HTES[k-1]) / zi
                             + Cp_w * ((mdot_CHP + mdot_HP_HT) * (T_HTES[k+1]-T_HTES[k])/2 
                                       + ca.fabs(mdot_CHP + mdot_HP_HT) * (T_HTES[k+1]-T_HTES[k])/2
                                       + (mdot_CHP + mdot_HP_HT - mdot_LOAD_H) * (T_HTES[k]-T_HTES[k-1])/2 
                                       + ca.fabs(mdot_CHP + mdot_HP_HT - mdot_LOAD_H)*(T_HTES[k-1]-T_HTES[k])/2)
                             - Aamb * kappa * (T_HTES[k] - T_amb_K)
                             + Pel_COIL_Nominal * COIL_Switch / nlayer_htes) )

# # combined for winter and summer (AdCM needs to be counted)
# a[k] = T_HTES[k+1]-T_HTES[k]
# b[k] = 0
# d[k] = T_HTES[k]-T_HTES[k-1]
# dxdt.append(1/(4.18 * mi) * ( Alayer * lambda_eff * (T_HTES[k+1] - 2*T_HTES[k] + T_HTES[k-1]) / zi
#                              + C1 * (a[k] + b[k])/2 + ca.fabs(C1)*(a[k] - b[k])/2
#                              + C2 * (b[k] + d[k])/2 + ca.fabs(C2)*(b[k] - d[k])/2
#                              - Aamb * kappa * (T_HTES[k] - T_amb_K)
#                              + Pel_COIL_Nominal * COIL_Switch / nlayer_htes) )


# Loop from the layer below the layer having output to load (index = layer_load_h-2), to the 2nd layer (index = 1):
# Use the formula above, with: x=mdot_CHP+mdot_HP_HT, y=mdot_LOAD_H, a=T_HTES[k+1]-T_HTES[k], b=T_HTES[k]-T_HTES[k-1]
# then the heat that the layer INCREASES from the liquid exchanging is:
#   (mdot_CHP + mdot_HP_HT - mdot_LOAD_H)*(T_HTES[k+1]-T_HTES[k]), if mdot_CHP + mdot_HP_HT > mdot_LOAD_H, i.e. water flowing downward
#   (mdot_CHP + mdot_HP_HT - mdot_LOAD_H)*(T_HTES[k]-T_HTES[k-1]), if mdot_CHP + mdot_HP_HT < mdot_LOAD_H, i.e. water flowing upward
for k in range(layer_load_h-2, 0, -1):
    dxdt.append(1/(4.18 * mi) * ( Alayer * lambda_eff * (T_HTES[k+1] - 2*T_HTES[k] + T_HTES[k-1]) / zi
                                 + 4.18 * ( (mdot_CHP + mdot_HP_HT - mdot_LOAD_H)*(T_HTES[k+1] - T_HTES[k-1])/2
                                           + ca.fabs(mdot_CHP + mdot_HP_HT - mdot_LOAD_H)*(T_HTES[k+1] - 2*T_HTES[k] + T_HTES[k-1])/2 )
                                 - Aamb * kappa * (T_HTES[k] - T_amb_K)
                                 + Pel_COIL_Nominal * COIL_Switch / nlayer_htes) )

# # combined for winter and summer (AdCM needs to be counted)
# for k in range(layer_load_h-2, 0, -1):
#     a[k] = T_HTES[k+1]-T_HTES[k]
#     b[k] = T_HTES[k]-T_HTES[k-1]
#     dxdt.append(1/(4.18 * mi) * ( Alayer * lambda_eff * (T_HTES[k+1] - 2*T_HTES[k] + T_HTES[k-1]) / zi
#                                  + C2 * (a[k] + b[k])/2 + ca.fabs(C2)*(a[k] - b[k])/2
#                                  - Aamb * kappa * (T_HTES[k] - T_amb_K)
#                                  + Pel_COIL_Nominal * COIL_Switch / nlayer_htes) )


# The bottom layer, which corresponds to T_HTES[0]:
# the heat that the layer RECEIVES from the liquid coming from the 2nd layer:
#   (mdot_CHP + mdot_HP_HT - mdot_LOAD_H)*T_HTES[1] > 0, if mdot_CHP + mdot_HP_HT>mdot_LOAD_H, i.e. receiving water flowing down
#   (mdot_CHP + mdot_HP_HT - mdot_LOAD_H)*T_HTES[0] < 0, if mdot_CHP + mdot_HP_HT<mdot_LOAD_H, i.e. losing water flowing up
# Note 14.3.2018: only in the bottom layer, there is coefficient 20.0 when exchanging with ambient temperature (because it touches the floor)

# old, seems wrong: dxdt.append(1/(4.18 * mi) * ( Alayer * lambda_eff * (T_HTES[1] - T_HTES[0]) / zi
#                              + 4.18 * ( mdot_LOAD_H*T_LOAD_H_FL - (mdot_CHP + mdot_HP_HT)*T_HTES[0]
#                                        + (mdot_CHP  + mdot_HP_HT - mdot_LOAD_H)*(T_HTES[1] + T_HTES[0])/2
#                                        + ca.fabs(mdot_CHP  + mdot_HP_HT - mdot_LOAD_H)*(T_HTES[1] - T_HTES[0])/2 )
#                              - 20.0 * Aamb * kappa * (T_HTES[0] - T_amb_K) ) )

dxdt.append(1/(4.18 * mi) * ( Alayer * lambda_eff * (T_HTES[1] - T_HTES[0]) / zi
                             + Cp_w * (mdot_LOAD_H * (T_LOAD_H_FL - T_HTES[0])
                                       + (mdot_CHP  + mdot_HP_HT - mdot_LOAD_H) * (T_HTES[1]-T_HTES[0])/2
                                       + ca.fabs(mdot_CHP  + mdot_HP_HT - mdot_LOAD_H)*(T_HTES[1] - T_HTES[0])/2 )
                             - 20.0 * Aamb * kappa * (T_HTES[0] - T_amb_K) ) )

# # # combined for winter and summer (AdCM needs to be counted)
# a[0] = T_HTES[1]-T_HTES[0]
# b[0] = 0
# dxdt.append(1/(4.18 * mi) * ( Alayer * lambda_eff * (T_HTES[1] - T_HTES[0]) / zi
#                              + Cp_w * mdot_LOAD_H * (T_LOAD_H_FL - T_HTES_b)
#                              + Cp_w * mdot_AdCM_HT * (T_AdCM_HT_FL - T_HTES_b)
#                              + C2 * (a[0] + b[0])/2 + ca.fabs(C2)*(a[0] - b[0])/2
#                              - 20.0 * Aamb * kappa * (T_HTES[0] - T_amb_K) ) )


dxdt = ca.vertcat(*dxdt)
#

## Algebraic equation, left hand side = 0
f_z = []
## Use equations like:
# f_z.append(P_th_CC/c_p - mdot_6_to_LOAD * (T_s[5] - T_LOAD_RL))
# f_z = ca.vertcat(*f_z)
#old: f_z.append(- v_dot_FUEL + 1000.0 * (Pel_CHP + Pth_CHP) / (853.5 * (CHP_eta_Thermal + CHP_eta_Electrical) * CHP_Fuel_LHV) )
# new 20.4.18
f_z.append(- vdot_FUEL + CHP_Switch * (1.8/1000.0))

#old: f_z.append(- v_dot_CHP + 0.433352645 + -0.01514531 * T_HTES_b + 0.00024329 * T_HTES_b ** 2.0)
# NOTE: to remove v_dot_CHP and m_dot_CHP_set, DONE
# new 20.4.18
f_z.append(- mdot_CHP + (rho_water/3600)*CHP_Switch* (0.433352645 - 0.01514531 * T_HTES_b + 0.00024329 * T_HTES_b ** 2.0))
f_z.append(- T_CHP_FL + T_HTES_b + (0.2392344497607656 * Pth_CHP / ((rho_water/3600)*(0.433352645 - 0.01514531 * T_HTES_b + 0.00024329 * T_HTES_b ** 2.0))) - 273.15)

#old: f_z.append(-Pth_CHP + (0.935962276499541 + 0.00389339241401334 * T_HTES_b + -5.18855946923975e-05 * T_HTES_b ** 2.0) * Pth_CHP_Nominal)
#old: f_z.append(-mdot_CHP + CHP_ON_int * m_dot_CHP_set)
f_z.append(-Pel_CHP + CHP_Switch * Pel_CHP_Nominal)
# NOTE: to remove CHP_ON_int, only use CHP_Switch, DONE

# old: f_z.append(- Pth_LOAD_H + 4.18 * m_dot_CC_H * (T_LOAD_H_CC_FL - T_LOAD_H_CC_RL) )
# old: f_z.append(- Pth_LOAD_H + 4.18 * (v_dot_LOAD_H*rho_water/3600) * ((T_HTES_l+273.15) - (T_LOAD_H_CC_RL+273.15)) )
# 4.18 = (1/Cp_w)
# old: f_z.append(-T_LOAD_H_FL + T_LOAD_H_CC_RL)
# new 20.4.18
f_z.append(-T_LOAD_H_FL + T_LOAD_H_CC_FL - (1/Cp_w)*Pth_LOAD_H/((rho_water/3600)*vdot_CC) - 273.15)
f_z.append(-mdot_LOAD_H + (1/Cp_w)*Pth_LOAD_H/(T_HTES_l - T_LOAD_H_FL))

#f_z.append(- COP + -0.049623287373 + 0.01893348591 * T_CTES_t + 0.013340776694 * T_AdCM_HT_FL + 0.017822939671 * T_AdCM_MT_RL + -0.001280352166 * T_CTES_t ** 2.0 + -0.000190832894 * T_AdCM_HT_FL ** 2.0 + -0.001993352016 * T_AdCM_MT_RL ** 2.0 + T_CTES_t * (-0.000334095159 * T_AdCM_HT_FL + 0.001455689548 * T_AdCM_MT_RL) + 0.000569253554 * T_AdCM_HT_FL * T_AdCM_MT_RL + 1.3421174e-05 * T_CTES_t * T_AdCM_HT_FL * T_AdCM_MT_RL)
#f_z.append(- Pth_AdCM_LT + AdCM_ON_int * (4.07950934099 + 0.04152472361 * T_CTES_t + 0.160630808297 * T_AdCM_HT_FL + -0.859860168466 * T_AdCM_MT_RL + 0.003462744142 * T_CTES_t ** 2.0 + -0.001049096999 * T_AdCM_HT_FL ** 2.0 + 0.015142231276 * T_AdCM_MT_RL ** 2.0 + T_CTES_t * (0.016955368833 * T_AdCM_HT_FL + -0.016151596215 * T_AdCM_MT_RL) + -0.001917799045 * T_AdCM_HT_FL * T_AdCM_MT_RL + -0.000200778961 * T_CTES_t * T_AdCM_HT_FL * T_AdCM_MT_RL))
#f_z.append(- Pth_AdCM_HT + Pth_AdCM_LT / COP - SF )

#f_z.append(-T_HX3_FL + T_HP_HT_FL - 3)
# NOTE: remove variable T_HX3_FL, DONE

# old, removed: f_z.append(-mdot_HP_HT + HP_Switch * m_dot_HP_HT_FL_Set)
# new 20.4.18
f_z.append(-mdot_HP_HT + (rho_water/3600)*HP_Switch * vdot_HP_HT1)
# NOTE: to remove m_dot_HP_HT_FL_Set, DONE
# old: f_z.append(-Pel_HP + HP_Switch * (1.367 + 0.0066*T_HP_MT_RL + 0.069*T_HP_HT_RL))
# new 20.4.18
f_z.append(-Pel_HP + HP_Switch * (1.367 + Pel_OC + 0.0066*(T_amb - Range_OC) + 0.069*T_HTES_b))
f_z.append(-Pth_HP_HT + HP_Switch * (13.3366 + 0.3852*(T_amb-Range_OC) - 0.042*T_HTES_b))

#f_z.append(-Pth_HP_MT + Pth_HP_HT - Pel_HP )
# NOTE: remove variable Pth_HP_MT, DONE

# old: f_z.append(-(T_HP_HT_FL+273.15) + (T_HP_HT_RL+273.15) + 0.2392344497607656 * Pth_HP_HT / m_dot_HP_HT_FL_Set)
# new 20.4.18
f_z.append(-T_HP_HT_FL - HX_loss + T_HTES_b + 0.2392344497607656 * Pth_HP_HT / (vdot_HP_HT1 * rho_water / 3600) - 273.15)
# old: f_z.append(-T_HTES_b + T_HP_HT_RL)


f_z = ca.vertcat(*f_z)


In [12]:
# For code testing
#for k in range(nlayer_htes-2, layer_load_h, -1):
#    print(k)

dxdt.shape
#f_z.shape

(10, 1)

In [13]:
# Cell 9

# Constraints

T_min_all = 0.0
T_max_all = 100.0

Pth_CHP_min = 0
Pth_CHP_max = Pth_CHP_Nominal

# for Winter problem
x_lbw = [Pth_CHP_min] + list(T_min_all*pl.ones(nlayer_htes))
x_ubw = [Pth_CHP_max] + list(T_max_all*pl.ones(nlayer_htes))

#zeros_z = list(0.0*pl.ones(f_z.numel()))
#zeros_x = list(0.0*pl.ones(dxdt.numel()))

In [14]:
nx
# dxdt.shape

10

In [15]:
f_z.shape

(10, 1)

In [16]:
# Cell 10

# Create dynamical model in form of a DAE (Differential Algebraic Equations) system

dae = {'x':x, 'z':y, 'p':u, 'ode':dxdt, 'alg':f_z}
#opts = {'tf':0.5} # interval length
I = ca.integrator('I','collocation', dae)

f = ca.Function('f', [x, y, u], [dxdt,f_z], ['x', 'z', 'p'], ['ode','alg'])
fz = ca.Function('fz', [y, x, u], [f_z], ['z','x', 'p'], ['alg'])
G1 = ca.rootfinder('G1','newton',fz)

# Test simulation
z_0_guess = [0.511 for i in range(ny)]
outz=G1(z=z_0_guess, x=x_0, p=u_0)
z_0_solve=outz['alg']

z_start=z_0_solve

r = I(x0=x_0,z0=z_start,p=u_0)
print(x_0)
print(r['xf'])

print(z_start)
print(r['zf'])

[1, 35.0, 33.75, 32.5, 31.25, 30.0, 28.75, 27.5, 26.25, 25.0]
[0.367879, 35.0005, 33.7505, 32.5005, 31.2505, 30.0005, 28.7505, 27.5005, 26.2505, 25.0091]
[0, 0, 0, -233.774, 0.000892663, -239.251, 0, 0, 0, -241.15]
[0, 0, 8.28509e-23, -236.54, 0.000892661, -239.251, 0, 0, 0, -241.149]


In [ ]:
# Cell 11

# Optimal control problem formulation
X = [ ca.MX.sym("x", nx) for i in range(N+1)]
U = [ ca.MX.sym("u", nu) for i in range(N)]
# X and U are lists of MX variables

# For testing, let the objective is to track the temperature of the top layer of the tank
T_ref = 70

# Multiple shooting

J = 0

X_next = []
Xs = []

gaps = []
g_min = []
g_max = []

lbw = [] # Lower bounds on variables
ubw = [] # Upper bounds on variables

# Generate the multiple shooting equality constraints, and set which
# optimization variables are discrete

discrete = []

V = []

V_init = []

z0 = z_0_solve

for k in range(N):
    r = I(x0=X[k],z0=z0,p=U[k])
    z0 = r["zf"]
    #X_next.append(r["xf"])
    gaps.append(X[k+1]-r["xf"])
    #Xs.append(X[k])
    
    V.append(X[k])
    V.append(U[k])
    
    g_min.append(np.zeros(nx))
    g_max.append(np.zeros(nx))
    discrete += [False] * nx # for states
    discrete += [True] * nu # true control is ON/OFF
    lbw += x_lbw
    lbw += u_lbw
    ubw += x_ubw
    ubw += u_ubw
    J = J + (r['xf'][-1]-T_ref)**2 # for the tracking objective, need to change to economic
    
    V_init.append(x_0)
    V_init.append(u_0)
    

V_init.append(x_0)    
    
V.append(X[-1])    
discrete += [False] * nx

lbw += x_lbw
ubw += x_ubw

u_true = ca.MX.sym("u",N)
p = ca.MX.sym("p",N)

# Stack the constraints as well as their bounds

gaps = ca.vertcat(*gaps)
g_min = ca.vertcat(*g_min)
g_max = ca.vertcat(*g_max)

p_track = Pth_Load_H_track

# Set up the NLP and the solver accordingly, and solve the optimization problem

nlp = {"x": ca.veccat(*V), "p": p, "f": J, "g": gaps}

## Solve MINLP with bonmin
# VERY time consuming
# opts = {"discrete": discrete}#, "bonmin":{"ipopt.print_level":5}}
# nlpsolver = ca.nlpsol("nlpsolver", "bonmin", nlp, opts) #, "bonmin.linear_solver": "ma86"})
# solution = nlpsolver(x0 = ca.vertcat(*V_init), p = p_track, lbx = lbw, ubx = ubw, lbg = g_min, ubg = g_max)

## Solve MINLP with KNITRO
# VERY time consuming
# opts = {"discrete": discrete}#, "bonmin":{"ipopt.print_level":5}}
# nlpsolver = ca.nlpsol("nlpsolver", "bonmin", nlp, opts) #, "bonmin.linear_solver": "ma86"})
# solution = nlpsolver(x0 = ca.vertcat(*V_init), p = p_track, lbx = lbw, ubx = ubw, lbg = g_min, ubg = g_max)
nlpsolver = ca.nlpsol('nlpsolver', 'knitro', nlp, {"discrete": discrete});
solution = nlpsolver(x0 = ca.vertcat(*V_init), p = p_track, lbx = lbw, ubx = ubw, lbg = g_min, ubg = g_max)


# # Relax MINLP to NLP, solve with IPOPT
# #opts = {"ipopt": {"hessian_approximation": "limited-memory", "linear_solver":"mumps"}}
# opts = {"ipopt": {"hessian_approximation": "limited-memory", "linear_solver":"ma97"}}
# solver = ca.nlpsol('solver','ipopt',nlp,opts)
# solution = solver(x0 = ca.vertcat(*V_init), p = p_track, lbx = lbw, ubx = ubw, lbg = g_min, ubg = g_max)

print(solution)


In [ ]:
len(lbw)

In [ ]:
# Cell 11

# Optimal control problem formulation
X = [ ca.MX.sym("x", nx) for i in range(N+1)]
U = [ ca.MX.sym("u", nu) for i in range(N)]
# X and U are lists of MX variables

# For testing, let the objective is to track the temperature of the top layer of the tank
T_ref = 70

# Multiple shooting

J = 0

X_next = []
Xs = []

gaps = []
g_min = []
g_max = []

lbw = [] # Lower bounds on variables
ubw = [] # Upper bounds on variables

# Generate the multiple shooting equality constraints, and set which
# optimization variables are discrete

discrete = []

V = []

V_init = []

z0 = z_0_solve

for k in range(N):
    r = I(x0=X[k],z0=z0,p=U[k])
    z0 = r["zf"]
    #X_next.append(r["xf"])
    gaps.append(X[k+1]-r["xf"])
    #Xs.append(X[k])
    
    V.append(X[k])
    V.append(U[k])
    
    g_min.append(np.zeros(nx))
    g_max.append(np.zeros(nx))
    discrete += [False] * nx # for states
    discrete += [True] * nu # true control is ON/OFF
    lbw += x_lbw
    lbw += u_lbw
    ubw += x_ubw
    ubw += u_ubw
    J = J + (r['xf'][-1]-T_ref)**2 # for the tracking objective, need to change to economic
    
    V_init.append(x_0)
    V_init.append(u_0)
    

V_init.append(x_0)    
    
V.append(X[-1])    
discrete += [False] * nx

lbw += x_lbw
ubw += x_ubw

u_true = ca.MX.sym("u",N)
p = ca.MX.sym("p",N)

# Stack the constraints as well as their bounds

gaps = ca.vertcat(*gaps)
g_min = ca.vertcat(*g_min)
g_max = ca.vertcat(*g_max)

p_track = Pth_Load_H_track

# Set up the NLP and the solver accordingly, and solve the optimization problem

nlp = {"x": ca.veccat(*V), "p": p, "f": J, "g": gaps}

## Solve MINLP with bonmin
# VERY time consuming
# opts = {"discrete": discrete}#, "bonmin":{"ipopt.print_level":5}}
# nlpsolver = ca.nlpsol("nlpsolver", "bonmin", nlp, opts) #, "bonmin.linear_solver": "ma86"})
# solution = nlpsolver(x0 = ca.vertcat(*V_init), p = p_track, lbx = lbw, ubx = ubw, lbg = g_min, ubg = g_max)

## Solve MINLP with KNITRO
# VERY time consuming
# opts = {"discrete": discrete}#, "bonmin":{"ipopt.print_level":5}}
# nlpsolver = ca.nlpsol("nlpsolver", "bonmin", nlp, opts) #, "bonmin.linear_solver": "ma86"})
# solution = nlpsolver(x0 = ca.vertcat(*V_init), p = p_track, lbx = lbw, ubx = ubw, lbg = g_min, ubg = g_max)
# nlpsolver = ca.nlpsol('nlpsolver', 'knitro', nlp, {"discrete": discrete});
# solution = nlpsolver(x0 = ca.vertcat(*V_init), p = p_track, lbx = lbw, ubx = ubw, lbg = g_min, ubg = g_max)


# # Relax MINLP to NLP, solve with IPOPT
# #opts = {"ipopt": {"hessian_approximation": "limited-memory", "linear_solver":"mumps"}}
opts = {"ipopt": {"hessian_approximation": "limited-memory", "linear_solver":"ma97"}}
solver = ca.nlpsol('solver','ipopt',nlp,opts)
solution = solver(x0 = ca.vertcat(*V_init), p = p_track, lbx = lbw, ubx = ubw, lbg = g_min, ubg = g_max)

print(solution)


In [18]:
len(lbw)

322

In [ ]:
# formulate the economic objective in optimization

sum_by_t(epex_price_buy[t]*P_buy_from_grid[t] + 

In [35]:
## Equations copied from Open Modelica initiated model, with variable names changed to Casadi name set
#560.794 * der(Pth_CHP) + Pth_CHP = Pth_CHP * CHP_ON_int 
v_dot_CHP = 0.433352645 + -0.01514531 * T_HTES_b + 0.00024329 * T_HTES_b ** 2.0
CHP_H_W_T_M_FL_K = (T_HTES_b+273.15) + 0.2392344497607656 * Pth_CHP / m_dot_CHP_set
v_dot_FUEL = 1000.0 * (Pel_CHP + Pth_CHP) / (853.5 * (CHP_eta_Thermal + CHP_eta_Electrical) * CHP_Fuel_LHV)
# begin missing eq for CHP
Pth_CHP = PartLoad_Ratio * Pth_CHP_Nominal;
PartLoad_Ratio = 0.935962276499541 + 0.00389339241401334 * CHP_H_W_T_M_RL + -5.18855946923975e-05 * CHP_H_W_T_M_RL ^ 2.0;
CHP_H_W_MF_M = 0.2761951388888889 * CHP_ON_int * CHP_H_W_VF_M;
CHP_H_E_PE_M = CHP_ON_int * Pel_CHP_Nominal "assuming electrical output is always constant";
# end missing eq for CHP

Pth_CC = 4.18 * m_dot_CC_H * (T_LOAD_H_CC_FL - T_LOAD_H_CC_RL)
Pth_CC = 4.18 * (v_dot_LOAD_H*rho_water/3600) * ((T_HTES_l+273.15) - (T_LOAD_H_CC_RL+273.15))

# begin missing eq for hot load
T_LOAD_H_FL = T_LOAD_H_CC_RL
# end missing eq for hot load

# begin missing eq for HX3
T_HX3_FL = T_HP_HT_FL - 3
# end missing eq for HX3

COP = -0.049623287373 + 0.01893348591 * T_CTES_t + 0.013340776694 * T_AdCM_HT_FL + 0.017822939671 * T_AdCM_MT_RL + -0.001280352166 * T_CTES_t ** 2.0 + -0.000190832894 * T_AdCM_HT_FL ** 2.0 + -0.001993352016 * T_AdCM_MT_RL ** 2.0 + T_CTES_t * (-0.000334095159 * T_AdCM_HT_FL + 0.001455689548 * T_AdCM_MT_RL) + 0.000569253554 * T_AdCM_HT_FL * T_AdCM_MT_RL + 1.3421174e-05 * T_CTES_t * T_AdCM_HT_FL * T_AdCM_MT_RL
Pth_AdCM_LT = AdCM_ON_int * (4.07950934099 + 0.04152472361 * T_CTES_t + 0.160630808297 * T_AdCM_HT_FL + -0.859860168466 * T_AdCM_MT_RL + 0.003462744142 * T_CTES_t ** 2.0 + -0.001049096999 * T_AdCM_HT_FL ** 2.0 + 0.015142231276 * T_AdCM_MT_RL ** 2.0 + T_CTES_t * (0.016955368833 * T_AdCM_HT_FL + -0.016151596215 * T_AdCM_MT_RL) + -0.001917799045 * T_AdCM_HT_FL * T_AdCM_MT_RL + -0.000200778961 * T_CTES_t * T_AdCM_HT_FL * T_AdCM_MT_RL)
Pth_AdCM_HT = Pth_AdCM_LT / COP - SF 
Pth_AdCM_MT = Pth_AdCM_HT + Pth_AdCM_LT 
T_AdCM_LT_FL_K = (T_CTES_t+273.15) + -0.2392344497607656 * Pth_AdCM_LT / m_dot_AdCM_LT_Set
T_AdCM_MT_FL_K = (T_AdCM_MT_RL+273.15) + 0.2392344497607656 * Pth_AdCM_MT / m_dot_AdCM_MT_Set
ADCM_C_W_T_M_HT_FL_K = (T_AdCM_HT_FL+273.15) + -0.2392344497607656 * Pth_AdCM_HT / m_dot_AdCM_HT_Set

Pth_HP_HT = HP_Switch * (9.0 + 0.294510922 * T_HP_MT_FL + T_HP_HT_RL * (0.064700246 + 0.002953381 * T_HP_MT_FL) + -0.001625553 * T_HP_MT_FL ** 2.0 + -0.001627312 * T_HP_HT_RL ** 2.0)
Pth_HP_MT = Pth_HP_HT - Pel_HP 
T_HP_HT_FL__K = (T_HP_HT_RL+273.15) + 0.2392344497607656 * Pth_HP_HT / m_dot_HP_HT_FL_Set
RevHP_HC_W_T_M_LT_FL__K = (T_HP_MT_RL+273.15) + -0.2392344497607656 * Pth_HP_MT / m_dot_HP_MT_FL_Set
Pel_HP = HP_Switch * (1.733202228 + -0.007333788 * T_HP_MT_FL + T_HP_HT_RL * (0.019283658 + 0.000450498 * T_HP_MT_FL) + -8.304799999999999e-05 * T_HP_MT_FL ** 2.0 + 0.000671146 * T_HP_HT_RL ** 2.0)

# begin missing eq for HP
T_HP_HT_RL = T_HTES_b
T_HP_MT_FL = T_HP_MT_RL + 5
# end missing eq for HP

Pth_CCM_LT = CCM_Switch * (9.0 + 0.308329176 * T_CCM_LT_FL + 0.045285097 * T_CCM_MT_RL + 0.002252906 * T_CCM_LT_FL * T_CCM_MT_RL + -0.001213212 * T_CCM_LT_FL ** 2.0 + -0.002264659 * T_CCM_MT_RL ** 2.0)
Pel_CCM = CCM_Switch * (1.833202228 + -0.007333788 * T_CCM_LT_FL + 0.019283658 * T_CCM_MT_RL + 0.000450498 * T_CCM_LT_FL * T_CCM_MT_RL + -8.304799999999999e-05 * T_CCM_LT_FL ** 2.0 + 0.000671146 * T_CCM_MT_RL ** 2.0)
Pth_CCM_MT = Pel_CCM + Pth_CCM_LT
RevHP_HC_W_T_M_LT_FL_K = (T_CCM_LT_FL+273.15) + -0.2392344497607656 * Pth_CCM_LT / m_dot_CCM_LT_FL_Set
T_CCM_MT_FL_K = (T_CCM_MT_RL+273.15) + 0.2392344497607656 * Pth_CCM_MT / m_dot_CCM_MT_FL_Set
Pth_CC = 4.18 * m_dot_CC_C * (T_LOAD_C_CC_RL - T_LOAD_C_CC_FL) 
Pth_CC = 4.18 * (v_dot_LOAD_C*rho_water/3600) * ((T_LOAD_C_CC_RL+273.15) - (T_LOAD_C_FL+273.15)) 
#4.18 * mi * der(HTES_H_W_T_M_IT_K[1]) = Alayer * lambda_eff * (HTES_H_W_T_M_IT_K[2] - HTES_H_W_T_M_IT_K[1]) / zi + 4.18 * (m_dot_LOAD * (T_HTES_LOAD_RL_K - HTES_H_W_T_M_IT_K[1]) + m_dot_AdCM_HT * (T_HTES_AdCM_RL_K - HTES_H_W_T_M_IT_K[1]) + d_pos * m_dot[2] * (HTES_H_W_T_M_IT_K[2] - HTES_H_W_T_M_IT_K[1])) + -20.0 * Aamb * kappa * (HTES_H_W_T_M_IT_K[1] - T_amb_K)
#4.18 * mi * der(HTES_H_W_T_M_IT_K[2]) = Alayer * lambda_eff * (HTES_H_W_T_M_IT_K[1] + -2.0 * HTES_H_W_T_M_IT_K[2] + HTES_H_W_T_M_IT_K[3]) / zi + 4.18 * m_dot[2] * (d_pos * (HTES_H_W_T_M_IT_K[3] - HTES_H_W_T_M_IT_K[2]) + d_neg * (HTES_H_W_T_M_IT_K[2] - HTES_H_W_T_M_IT_K[1])) + COIL_H_E_PT_M / /*Real*/(n) - Aamb * kappa * (HTES_H_W_T_M_IT_K[2] - T_amb_K)
#Chot = 3.66736 * (v_dot_OC1*rho_water/3600)
#Ccold = 1.005 * (v_dot_air_OC1*rho_water/3600)
#Cmin = min(Chot, Ccold)
#Cmax = max(Chot, Ccold)
#qmax = Cmin * (T_OC1_RL - T)
#q = eff * qmax
#eff = (1.0 - exp(NTU * (Cr - 1.0))) / (1.0 - Cr * exp(NTU * (Cr - 1.0)))
#RPM_max / RPM_real = Volt_max / Volt_real
#Pel_max / Pel_OC1 = (RPM_max / RPM_real) ** 3.0
#v_dot_air_max / v_dot_air_real = RPM_max / RPM_real
#Chot = 3.66736 * (v_dot_OC2*rho_water/3600)
#Ccold = 1.005 * (v_dot_air_OC2*rho_water/3600)
#Cmin = min(Chot, Ccold)
#Cmax = max(Chot, Ccold)
#qmax = Cmin * (T_OC2_RL - T)
#q = eff * qmax
#eff = (1.0 - exp(NTU * (Cr - 1.0))) / (1.0 - Cr * exp(NTU * (Cr - 1.0)))
#RPM_max / RPM_real = Volt_max / Volt_real
#Pel_max / Pel_OC2 = (RPM_max / RPM_real) ** 3.0
#v_dot_air_max / v_dot_air_real = RPM_max / RPM_real
#Chot = 4.18 * m_dot_OC
#Ccold = 1.005 * m_dot_air
#Cmin = min(Chot, Ccold)
#Cmax = max(Chot, Ccold)
#qmax = Cmin * (T_CCM_MT_RL - T)
#q = eff * qmax
#eff = (1.0 - exp(NTU * (Cr - 1.0))) / (1.0 - Cr * exp(NTU * (Cr - 1.0)))
#q = Chot * (T_CCM_MT_RL - T_CCM_MT_FL)
#q = Ccold * (T_air_out - T)
#RPM_max / RPM_real = Volt_max / Volt_real
#Pel_max / Pel_OC3 = (RPM_max / RPM_real) ** 3.0
#v_dot_air_max / v_dot_air_real = RPM_max / RPM_real
#4.18 * mi * der(CTES_H_W_T_M_IT_K[1]) = 4.18 * (m_dot_AdCM * (T_CTES_AdCM_In_K - CTES_H_W_T_M_IT_K[1]) + m_dot_RevHP * (T_CTES_RevHP_In_K - CTES_H_W_T_M_IT_K[1])) + Alayer * lambda_eff * (CTES_H_W_T_M_IT_K[2] - CTES_H_W_T_M_IT_K[1]) / zi + 4.18 * d_neg * m_dot[2] * (CTES_H_W_T_M_IT_K[1] - CTES_H_W_T_M_IT_K[2]) - Aamb * kappa * (CTES_H_W_T_M_IT_K[1] - T_amb_K)
#4.18 * mi * der(CTES_H_W_T_M_IT_K[2]) = Alayer * lambda_eff * (CTES_H_W_T_M_IT_K[1] + -2.0 * CTES_H_W_T_M_IT_K[2] + CTES_H_W_T_M_IT_K[3]) / zi + 4.18 * m_dot[2] * (d_neg * (CTES_H_W_T_M_IT_K[2] - CTES_H_W_T_M_IT_K[3]) + d_pos * (CTES_H_W_T_M_IT_K[1] - CTES_H_W_T_M_IT_K[2])) - Aamb * kappa * (CTES_H_W_T_M_IT_K[2] - T_amb_K)


NameError: name 'T_CTES_t' is not defined